# Quadratic Constraints

This example demonstrates how to create and solve models with quadratic constraints (QCP/QCQP). A quadratic constraint has the form:

$$ x^T Q x + a^T x \leq b $$

In [ ]:
import linopy
import numpy as np

## Setting up the Model

We'll maximize $x + 2y$ subject to a linear constraint and a quadratic constraint (a circle).

In [ ]:
m = linopy.Model()

x = m.add_variables(lower=0, upper=10, name="x")
y = m.add_variables(lower=0, upper=10, name="y")

## Adding Constraints

Linear constraints work as usual:

In [ ]:
m.add_constraints(x + y <= 8, name="budget")

Quadratic constraints use `add_quadratic_constraints`. Here we constrain $(x, y)$ to lie within a circle of radius 5:

In [ ]:
m.add_quadratic_constraints(x * y + y * y, "<=", 25, name="circle")

You can also have mixed terms like $xy$:

In [ ]:
m.add_quadratic_constraints(x * y, "<=", 10, name="mixed")

## Objective and Solve

In [ ]:
m.add_objective(x + 2 * y, sense="max")

In [ ]:
m.type

In [ ]:
m.solve(solver_name="gurobi")

## Results

In [ ]:
x.solution

In [ ]:
y.solution

In [ ]:
m.objective.value

## Inspecting Quadratic Constraints

In [ ]:
m.quadratic_constraints

In [ ]:
m.quadratic_constraints["circle"]

## Multi-dimensional Quadratic Constraints

Quadratic constraints also work with coordinates, just like linear constraints:

In [ ]:
m2 = linopy.Model()

# Variables with a dimension
x = m2.add_variables(lower=0, coords=[range(3)], name="x")
y = m2.add_variables(lower=0, coords=[range(3)], name="y")

# This creates 3 quadratic constraints, one for each coordinate
m2.add_quadratic_constraints(x * x * np.array([1, 1.1, 1.2]) + y * y, "<=", np.array([20, 25, 30]), name="circles")

In [ ]:
m2.quadratic_constraints["circles"]